In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('signal_feature_new.csv')
del df['Unnamed: 0']

In [3]:
df.head()

,signal,path,mfcc,mfcc_mean,zcr,sc,rms,melspectrogram
0,2,F:\RMS Samples\S2\A17BA_12055_1333_27_08_2017.MP3,"[-1.19005562e+02,2.06678009e+02,-1.05538704e+0...",0.055562,0.114928,1463.040193,0.145157,4.540862
1,2,F:\RMS Samples\S2\A17BA_12055_1336_27_08_2017.MP3,"[-1.1469097e+02,2.1500935e+02,-1.1449629e+02,4...",0.384692,0.119250,1484.006563,0.147916,4.452103
2,2,F:\RMS Samples\S2\A17BA_12055_1341_27_08_2017.MP3,"[-1.22675224e+02,2.07198257e+02,-1.02659149e+0...",0.464670,0.109778,1413.981019,0.134147,3.892156
3,2,F:\RMS Samples\S2\A17BA_12055_1346_27_08_2017.MP3,"[-1.1597163e+02,2.1474585e+02,-1.1746875e+02,4...",0.278849,0.119023,1495.144218,0.147518,4.409816
4,2,F:\RMS Samples\S2\A17BA_12055_1351_27_08_2017.MP3,"[-1.04732437e+02,2.15139145e+02,-1.20868294e+0...",0.233441,0.125884,1538.705541,0.154144,4.562316


In [4]:
def quality(signal):
    if signal == 1 or signal == 2:
        return 0
    else:
        return 1

In [5]:
#S=1 & 2 will be 0 {Bad} and rest 1 {Good}
df['quality'] = df['signal'].apply(quality)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407 entries, 0 to 406
Data columns (total 9 columns):
signal            407 non-null int64
path              407 non-null object
mfcc              407 non-null object
mfcc_mean         407 non-null float64
zcr               407 non-null float64
sc                407 non-null float64
rms               407 non-null float64
melspectrogram    407 non-null float64
quality           407 non-null int64
dtypes: float64(5), int64(2), object(2)
memory usage: 28.7+ KB


In [7]:
df.head()

,signal,path,mfcc,mfcc_mean,zcr,sc,rms,melspectrogram,quality
0,2,F:\RMS Samples\S2\A17BA_12055_1333_27_08_2017.MP3,"[-1.19005562e+02,2.06678009e+02,-1.05538704e+0...",0.055562,0.114928,1463.040193,0.145157,4.540862,0
1,2,F:\RMS Samples\S2\A17BA_12055_1336_27_08_2017.MP3,"[-1.1469097e+02,2.1500935e+02,-1.1449629e+02,4...",0.384692,0.119250,1484.006563,0.147916,4.452103,0
2,2,F:\RMS Samples\S2\A17BA_12055_1341_27_08_2017.MP3,"[-1.22675224e+02,2.07198257e+02,-1.02659149e+0...",0.464670,0.109778,1413.981019,0.134147,3.892156,0
3,2,F:\RMS Samples\S2\A17BA_12055_1346_27_08_2017.MP3,"[-1.1597163e+02,2.1474585e+02,-1.1746875e+02,4...",0.278849,0.119023,1495.144218,0.147518,4.409816,0
4,2,F:\RMS Samples\S2\A17BA_12055_1351_27_08_2017.MP3,"[-1.04732437e+02,2.15139145e+02,-1.20868294e+0...",0.233441,0.125884,1538.705541,0.154144,4.562316,0


In [8]:
df['quality'].value_counts()

1    205
0    202
Name: quality, dtype: int64

In [9]:
P_0 = 203/(202+205)
P_1 = 205/(202+205)

In [10]:
entropy_quality = -( P_0*np.log2(P_0) + P_1*np.log2(P_1) )

In [11]:
print(entropy_quality)

0.9988905700084068


In [19]:
import scipy.stats as st
#Entropy of Quality
E_Quality = st.entropy([205,202],base=2)
print(E_Quality)

0.9999608075721955


In [20]:
df['signal'].value_counts()

4    105
1    102
3    100
2    100
Name: signal, dtype: int64

In [21]:
#Entropy of (Quality,Signal)
p_s1 = 102/407
p_s2 = 100/407
p_s3 = 100/407
p_s4 = 105/407

E_Quality_Signal = (p_s1*st.entropy([102,202],base=2) + p_s2*st.entropy([100,202],base=2)
                    +p_s3*st.entropy([100,205],base=2)+
                    p_s4*st.entropy([105,205],base=2))
print(E_Quality_Signal)

0.9182916130667387


In [23]:
#Information Gain of Quality and Signal
Gain_Quality_Signal = E_Quality - E_Quality_Signal
print(Gain_Quality_Signal)

0.08166919450545684


# Classifier

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X = df[['mfcc_mean','zcr','sc','rms','melspectrogram']]
y = df['signal']
X_train,X_test,y_train,y_test = train_test_split(X_new,y,random_state=123,test_size=0.33)

In [66]:
clf = RandomForestClassifier(n_estimators=30,max_depth=5)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [67]:
pred = clf.predict(X_test)

print(clf.score(X_train,y_train))
print(clf.score(X_test,y_test))

0.7610294117647058
0.5777777777777777


In [68]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,pred))

[[19  4  0  5]
 [ 7 21  1  1]
 [ 2 13 11 12]
 [ 3  7  2 27]]


In [69]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           1       0.61      0.68      0.64        28
           2       0.47      0.70      0.56        30
           3       0.79      0.29      0.42        38
           4       0.60      0.69      0.64        39

    accuracy                           0.58       135
   macro avg       0.62      0.59      0.57       135
weighted avg       0.63      0.58      0.56       135



In [70]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaled_data = scaler.transform(X)


In [37]:
X_new = scaled_data.tolist()

# testing other

In [68]:
df = pd.read_csv('MFCC_Break.csv')

In [69]:
df.head()

,signal,a,b,c,d,e,f,g,h,i,...,ae,af,ag,ah,ai,aj,ak,al,am,an
0,2,-119.0,207.0,-106.0,44.8,11.20,-19.3,10.7,-6.19,6.10,...,0.236,-0.803,-2.830,0.675,-3.38,-1.7500,-1.450,-2.96,0.0204,-2.04
1,2,-115.0,215.0,-114.0,49.5,9.32,-19.2,12.0,-8.52,8.62,...,0.831,0.178,-1.510,1.640,-2.60,-0.1160,-0.789,-2.95,0.4730,-1.89
2,2,-123.0,207.0,-103.0,41.4,14.50,-14.3,10.2,-3.70,6.08,...,1.800,0.663,-1.250,1.300,-3.04,-0.0224,-0.860,-2.83,0.3950,-2.65
3,2,-116.0,215.0,-117.0,47.1,10.40,-19.1,12.3,-8.26,7.51,...,0.975,0.636,-0.591,2.110,-2.32,0.7450,-0.106,-2.73,1.1700,-1.89
4,2,-105.0,215.0,-121.0,50.5,9.81,-19.8,15.7,-10.30,8.66,...,-1.910,-0.794,-3.110,0.124,-3.68,-1.3900,-0.231,-3.06,1.2600,-0.77


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407 entries, 0 to 406
Data columns (total 41 columns):
signal    407 non-null int64
a         407 non-null float64
b         407 non-null float64
c         407 non-null float64
d         407 non-null float64
e         407 non-null float64
f         407 non-null float64
g         407 non-null float64
h         407 non-null float64
i         407 non-null float64
j         407 non-null float64
k         407 non-null float64
l         407 non-null float64
m         407 non-null float64
n         407 non-null float64
o         407 non-null float64
p         407 non-null float64
q         407 non-null float64
r         407 non-null float64
s         407 non-null float64
t         407 non-null float64
u         407 non-null float64
v         407 non-null float64
w         407 non-null float64
x         407 non-null float64
y         407 non-null float64
z         407 non-null float64
aa        407 non-null float64
ab        407 non-null float6

In [71]:
X = df[['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','aa','ab','ac','ad','ae','af','ag','ah','ai','aj','ak','al','am','an']]
y = df['signal']

In [72]:
X

,a,b,c,d,e,f,g,h,i,j,...,ae,af,ag,ah,ai,aj,ak,al,am,an
0,-119.0,207.0,-106.0,44.8,11.200,-19.30,10.70,-6.19,6.10,-10.00,...,0.236,-0.803,-2.830,0.675,-3.38,-1.7500,-1.450,-2.96,0.0204,-2.04
1,-115.0,215.0,-114.0,49.5,9.320,-19.20,12.00,-8.52,8.62,-10.90,...,0.831,0.178,-1.510,1.640,-2.60,-0.1160,-0.789,-2.95,0.4730,-1.89
2,-123.0,207.0,-103.0,41.4,14.500,-14.30,10.20,-3.70,6.08,-12.40,...,1.800,0.663,-1.250,1.300,-3.04,-0.0224,-0.860,-2.83,0.3950,-2.65
3,-116.0,215.0,-117.0,47.1,10.400,-19.10,12.30,-8.26,7.51,-11.00,...,0.975,0.636,-0.591,2.110,-2.32,0.7450,-0.106,-2.73,1.1700,-1.89
4,-105.0,215.0,-121.0,50.5,9.810,-19.80,15.70,-10.30,8.66,-8.11,...,-1.910,-0.794,-3.110,0.124,-3.68,-1.3900,-0.231,-3.06,1.2600,-0.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,-162.0,214.0,-81.8,49.3,9.500,-9.74,3.19,-6.98,9.27,-15.60,...,-6.850,-1.590,-3.520,-2.510,1.52,-1.7800,1.380,-2.03,-3.9500,-4.24
403,-133.0,223.0,-132.0,61.7,0.656,-16.30,18.00,-14.80,13.80,-11.30,...,-0.480,-1.870,-2.900,1.050,-4.46,0.1450,-0.980,-2.51,0.8660,-3.03
404,-136.0,230.0,-142.0,66.1,-3.390,-16.90,19.60,-18.00,14.90,-12.10,...,1.390,-1.040,-2.320,2.310,-3.33,1.3300,-0.384,-2.21,1.3000,-2.66
405,-135.0,219.0,-128.0,57.2,3.190,-15.60,15.70,-12.60,12.80,-12.30,...,0.584,-0.829,-1.990,1.710,-2.96,0.9640,-0.735,-2.55,1.1400,-2.49


In [156]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=123,test_size=0.33)

In [157]:
clf = RandomForestClassifier(n_estimators=40,max_depth=20)
clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [158]:
pred = clf.predict(X_test)

print(clf.score(X_train,y_train))
print(clf.score(X_test,y_test))

0.9926470588235294
0.6148148148148148


In [161]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(20, ), activation='relu', solver='adam', alpha=0.0001,
                    batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5,
                    max_iter=1500, shuffle=True, random_state=100, tol=0.0001, verbose=False, warm_start=False, 
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9,
                    beta_2=0.999, epsilon=1e-08, n_iter_no_change=10)

In [163]:
clf.fit(X_train,y_train)
pred = clf.predict(X_test)

print(clf.score(X_train,y_train))
print(clf.score(X_test,y_test))

0.7352941176470589
0.6370370370370371
